In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

In [2]:
import re
pat = re.compile('[ก-๙]') # คำไทย คือมีอักษรไทย อยู่ในช่วงประมาณนี้
# cr Cr. https://colab.research.google.com/drive/1SdaxK-jOOZDWS-2z_0RoYVuSdnr_Ca6B#scrollTo=fWq7xXOAPz6_

In [3]:
encode = tokenizer.encode
decode = tokenizer.decode
tokenize = tokenizer.tokenize

In [4]:
vocab=tokenizer.vocab

In [5]:
all_vocabs = []
for v in vocab.values():
  k = decode(v)
  all_vocabs.append(k)

In [6]:
thais = [(i,v) for i,v in enumerate(all_vocabs) if pat.search(v)]
len(thais)

2569

In [7]:
thais[0:10]

[(25, 'ฝั'),
 (112, 'เริ่ม'),
 (175, 'แนะนำ'),
 (262, 'แท'),
 (321, 'วิเคราะ'),
 (349, 'บริษัท'),
 (350, 'ไก่'),
 (354, 'เรียน'),
 (368, 'ฌ'),
 (407, 'เบื้อง')]

In [8]:
sorted=[]
with open("rwkv_vocab_v20230424.txt", "r") as f:
    for l in f.readlines():
        x = eval(l[l.index(' '):l.rindex(' ')])
        x = x.encode("utf-8") if isinstance(x, str) else x
        sorted += [x]

In [10]:
sorted[1000]

b'Et'

In [14]:
# latest thai token
sorted[58513].decode("utf-8")

'ที่'

In [16]:
def thai_search(i):
    try:
        return pat.search(i.decode("utf-8"))
    except:
        return False

In [17]:
rwkv_thais = [i.decode("utf-8") for i in sorted if thai_search(i)]

In [27]:
len(rwkv_thais)

59

In [19]:
"ฅ" in rwkv_thais

False

In [20]:
"๖" in rwkv_thais

False

In [21]:
thai_set_rwkv=set(rwkv_thais)

In [30]:
from pythainlp import thai_characters

In [31]:
add_thai_token=[]
for i in thai_characters:
    if i not in thai_set_rwkv:
        add_thai_token.append(i.encode("utf-8") if isinstance(i, str) else i)

In [32]:
len(add_thai_token)

40

In [33]:
for _,i in thais:
    if i not in thai_set_rwkv:
        add_thai_token.append(i.encode("utf-8") if isinstance(i, str) else i)

In [34]:
len(add_thai_token)

2551

In [35]:
add_thai_token[0]

b'\xe0\xb8\x83'

In [36]:
new_sorted=sorted+add_thai_token

In [37]:
len(new_sorted)

68080

In [40]:
new_sorted[0:5]

[b'\x00', b'\x01', b'\x02', b'\x03', b'\x04']

In [41]:
rwkv_sorted = [(i,len(i)) for  i in new_sorted]

In [42]:
rwkv_sorted[0:10]

[(b'\x00', 1),
 (b'\x01', 1),
 (b'\x02', 1),
 (b'\x03', 1),
 (b'\x04', 1),
 (b'\x05', 1),
 (b'\x06', 1),
 (b'\x07', 1),
 (b'\x08', 1),
 (b'\t', 1)]

In [43]:
rwkv_sorted.sort(key=lambda tup: tup[1])

In [47]:
with open("rwkv_vocab_v20250216.txt","w") as f:
    for index,(k,l) in enumerate(rwkv_sorted):
        f.write(f"{index} {k} {l}\n")